Этот ноутбук содержит реализацию метода Шепли-Сноу для некоторых задач из области игр поиска.

Метод Шепли-Сноу используется для нахождения оптимальных стратегий и значения игры в прямоугольных играх (антагонистических играх размера $n \times m$). Поскольку рассматриваемые игры поиска как раз относятся к играм такого типа, данный метод подходит здесь идеально.

Задача, решённая в данном ноутбуке, может быть сформулирована следующим образом.

Пусть у нас есть два игрока — $H$ и $S$, один объект и $n$ ячеек, куда этот объект можно положить. Игрок $H$ прячет этот объект в одну из $n$ ячеек, а игрок $S$ пытается за $m$ попыток его найти. Вероятность найти объект в ячейке, если он там содержится, не 100%-ая и зависит от ячейки, в которой объект спрятан. Игрок $S$ может смотреть в одну и ту же ячейку дважды.

Цель игрока $H$ — спрятать объект в такую ячейку, чтобы вероятность обнаружения объекта игроком $S$ была минимальна.

Цель игрока $S$ — найти объект или максимально (с точки зрения вероятности обнаружения) приблизиться к этому.

Данная игра, как и любая матричная игра, может быть представлена в виде матрицы игры — таблицы, где строки отвечают за стратегии одного игрока ($S$ в данном случае), а столбцы — за стратегии другого игрока (соответственно, $H$).

На пересечении $i$-ой строки и $j$-ого столбца, в ячейке $(i, j)$ матрицы игры, находится выигрыш (проигрыш) игрока $S$ (игрока $H$) — вероятность того, что игрок $S$ найдёт объект в данной ячейке.

Ниже находится необходимые для работы данного ноутбука технические детали — импорты, реализация необходимых функций (построение матрицы игры, сам метод Шепли-Сноу и пр.). После этого будет представлен относительно дружелюбный интерфейс для взаимодействия с ноутбуком, где пользователю будет предложено ввести необходимые входные параметры, после чего он получит результат — матрицу игры, оптимальные стратегии и значение игры для данного входа.

Наконец отметим, что если алгоритм вывел несколько пар оптимальных стратегий, то их следует воспринимать как крайние точки симплекса, все значения которого являются парами оптимальных стратегий.

Необходимые импорты:

In [1]:
import numpy as np
import math
import itertools
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import time

Используемые функции:

In [2]:
# finding nonnegative integers p1, p2, ..., pn such that p1 + p2 + ... + pn = m
# m should be a nonnegative integer
# n should be a positive integer
# remainder should ALWAYS be empty list (except for internal recursive function calls)
# return all possible composition of m into n parts
def composition(m, n, remainder = []):
    if n < 1:
        raise ValueError("n must be positive integer!")
    elif n == 1:
        return [remainder + [m]]
    else:
        answer = []
        for i in range(m, -1, -1):
            answer += composition(m-i, n-1, remainder+[i])
    return answer

In [3]:
def get_game_matrix(n, m, p, eps=1e-6):
    if not isinstance(n, int) or n < 1:
        raise ValueError("n must be positive integer!")
    if not isinstance(m, int) or m < 1:
        raise ValueError("m must be positive integer!")
    if len(p) != n:
        raise ValueError("Size of p must be equal to n!")
    for i in range(n):
        if p[i] < eps or p[i] > 1 - eps:
            raise ValueError("Probability must be between 0 and 1 (ends are not included)!")
    X = np.tile(np.array(p), (math.comb(m+n-1, m), 1))

    searchings = composition(m, n, [])

    if X.shape[0] != len(searchings) or X.shape[1] != len(searchings[0]):
        raise ValueError("Something went wrong: Matrix of game and Searchings matrix have"
                         "different shapes!")

    #new probabilities in a cell by number of searchings there 
    prob_by_searchings = np.tile(np.array(p).reshape(-1, 1), (1, m))
    for j in range(1, m):
        for i in range(len(p)):
            prob_by_searchings[i][j] = (prob_by_searchings[i][j - 1] + prob_by_searchings[i][0]
                                        * (1 - prob_by_searchings[i][0]) ** j)

    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            X[i][j] = prob_by_searchings[j][searchings[i][j] - 1] if searchings[i][j] > 0 else 0
    return X, searchings

In [24]:
def adjoint(matrix, det):
    if det != 0:
        return np.linalg.inv(matrix) * det
    else:
        C = np.zeros(matrix.shape)
        nrows, ncols = C.shape
        for row in range(nrows):
            for col in range(ncols):
                minor = np.delete(np.delete(matrix, row, 0), col, 1)
                C[row][col] = (-1)**(row+col) * np.linalg.det(minor)
        return C

In [19]:
def find_equilibrium(X, detailed_feedback=False, eps=1e-6):
    total_counter = 0
    answer_strategies = []
    answer_value = None
    
    min_size = min(X.shape[0], X.shape[1]) + 1
    max_value = 0
    for i in range(1, min_size):
        max_value += math.comb(X.shape[0], i) * math.comb(X.shape[1], i)
    f = IntProgress(min=1, max=max_value) # instantiate the bar
    display(f)
    
    for i in range(1, min_size):
        for a in list(itertools.combinations(range(X.shape[0]), i)):
            for b in list(itertools.combinations(range(X.shape[1]), i)):
                try:
                    total_counter += 1
                    f.value += 1 # signal to increment the progress bar
                    # time.sleep(.1)
                    B = X[np.ix_(a,b)]
                    
                    if detailed_feedback:
                        print(f"Case #{total_counter}")
                        print(f"Square submatrix for rows {a} and cols {b}:")
                        print(B)
                    
                    J = np.ones((1, i))
                    detB = np.linalg.det(B)
                    adjB = adjoint(B, detB)
                    xnom = J.dot(adjB)
                    denom = xnom.dot(J.transpose())[0][0]
                    if abs(denom) < eps:
                        raise Exception("Denominator is zero!")
                    ynom = J.dot(adjB.transpose())
                    x = np.zeros(X.shape[0])
                    x[np.ix_(a)] = xnom / denom
                    y = np.zeros(X.shape[1])
                    y[np.ix_(b)] = ynom / denom
                    v = detB / denom
                    if np.any((x < -eps)) or np.any((y < -eps)):
                        raise Exception("Some of probabilities are negative!")
                    for j in range(X.shape[1]):
                        expectation = 0
                        for k in range(len(x)):
                            expectation += x[k] * X[k][j]
                        if (expectation > v + eps and y[j] > eps or expectation < v - eps or
                                y[j] > eps and abs(expectation - v) > eps):
                            raise Exception("Check failed!")
                    for k in range(X.shape[0]):
                        expectation = 0
                        for j in range(len(y)):
                            expectation += y[j] * X[k][j]
                        if (expectation < v - eps and x[k] > eps or expectation > v + eps or
                                x[k] > eps and abs(expectation - v) > eps):
                            raise Exception("Check failed!")
                    
                    if detailed_feedback:
                        print(f"Successful! Optimal strategies were found!")
                        print(f"x = {[round(k, 5) for k in x]}")
                        print(f"y = {[round(k, 5) for k in y]}")
                        print(f"v = {round(v, 5)}")
                        print()
                        
                    if not answer_value:
                        answer_value = v
                    elif abs(answer_value - v) > eps:
                        raise RuntimeError("Game values are different for different pair of"
                                           "strategies!")
                    answer_strategies.append((x, y))
                except Exception as e:
                    if detailed_feedback:
                        print(e)
                        print()
        
    return answer_value, answer_strategies

In [14]:
def ordinal(n):
    return "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])

In [15]:
def print_game_matrix(X, rows, columns):
    df = pd.DataFrame(X, index=rows, columns=columns)
    print("Matrix of game:")
    display(df)
    print()

In [30]:
def print_optimal_strategies(x, y, rows, columns):
    dfx = pd.DataFrame(np.array([round(e, 5) for e in x]).reshape(1, -1), columns=rows, index=['x'])
    dfy = pd.DataFrame(np.array([round(e, 5) for e in y]).reshape(1, -1), columns=columns, index=['y'])
    display(dfx.loc[:, [not (dfx[e] == 0).all() for e in dfx.columns]])
    display(dfy.loc[:, [not (dfy[e] == 0).all() for e in dfy.columns]])

Параметры, используемые в методе:

$n$ — количество ячеек всего, $n \in \mathbb N$;

$m$ — количество ячеек, которые может осмотреть игрок $S$, $m \in \mathbb N$;

$p = (p_1, ..., p_n)$ — вероятности найти объект в соответствующих ячейках, $p_i \in (0; 1)$.

Введите входные данные ниже:

In [21]:
n = 3
m = 5
p = [0.2, 0.4, 0.8]

Матрица игры, оптимальные стратегии и соответствующее им значение игры:

In [32]:
try:
    X, searchings = get_game_matrix(n, m, p)
    rows = ['{' + ', '.join(str(e) for e in x) + '}' for x in searchings]
    columns = [ordinal(x) for x in range(1, X.shape[1] + 1)]
    print_game_matrix(X, [x + " times" for x in rows], [y + " cells" for y in columns])
    value, strategies = find_equilibrium(X)
    print(f"Value of the game is {round(value, 5)}.")
    print()
    print("Optimal strategies (only non-zero values!):")
    print()
    counter = 0
    for s in strategies:
        counter += 1
        print(f"Pairs of optimal strategies #{counter}:")
        print_optimal_strategies(s[0], s[1], rows, columns)
except ValueError as e:
    print(f"Error! {e}")

Matrix of game:


,1st cells,2nd cells,3rd cells
"{5, 0, 0} times",0.67232,0.00000,0.00000
"{4, 1, 0} times",0.59040,0.40000,0.00000
"{4, 0, 1} times",0.59040,0.00000,0.80000
"{3, 2, 0} times",0.48800,0.64000,0.00000
"{3, 1, 1} times",0.48800,0.40000,0.80000
"{3, 0, 2} times",0.48800,0.00000,0.96000
"{2, 3, 0} times",0.36000,0.78400,0.00000
"{2, 2, 1} times",0.36000,0.64000,0.80000
"{2, 1, 2} times",0.36000,0.40000,0.96000
"{2, 0, 3} times",0.36000,0.00000,0.99200


IntProgress(value=1, max=2023, min=1)

Value of the game is 0.48954.

Optimal strategies (only non-zero values!):

Pairs of optimal strategies #1:


,"{4, 1, 0}","{3, 2, 0}","{3, 1, 1}"
x,0.01501,0.37307,0.61192


,1st,2nd,3rd
y,0.64322,0.27444,0.08233
